### ah_value - Stateful Variables

The AH Value class is a simple, encapsulated c++ class to help track values.  It is written with the signal-slot paradigm so that events can be triggered as the values change.  This is a way to decrease the overhead that must happen to trace some values (such as checking if something is over a certain value every loop).

#### Usage

Lets say we want to create a program that indicates whether a motor is up to
speed, and thereafter, how much it has varied from that speed.  We would write a
notifier class, as below.

In [5]:
from cppm import cppmagic

In [14]:
%%include notifier
#include "../common/sigslot.h"
#include "../ah_value.h"
#include <stdio.h>

class notifier : public sigslot::has_slots<>{
// we have to make this a sigslot so that it can accept slots
public:
    // a value object for our notifier class
    value<int> *vl;
    // a constructor to pass in an external notifier class
    notifier(value<int>*);
    // we need a slot that will tell when it's up to speed
    void up_to_speed(void);
};

notifier::notifier(value<int> *_vl){
    vl = _vl;
}

void notifier::up_to_speed(void){
    // let us know that we're up to speed
    printf("Motor is up to speed\n");
    // now reset the max min so we only get the maximum and minimum when up to
    // speed
    vl->resetmaxmin();
}

In [15]:
%set_compiler g++ --std=c++11 -Wall

In [16]:
%%runcpp test_value
#include "../ah_value.h"
#include "notifier.h"

int main(void){
    // we need to create a new value class of type int
    value<int> *vl = new value<int>();
    // we need to create a new notifier class and pass in our value
    notifier *ntfr = new notifier(vl);
    // we need to connect the signal for above level to the slot telling us
    // we're up to speed
    vl->abovelevel.connect(&ntfr, notifier::up_to_speed);
    // now we have to set what level we're up to speed at
    vl->setlevel(5);
    // now we want to slowly increase the speed
    vl->set(0);
    vl->set(1);
    vl->set(3);
    vl->set(5);
    vl->set(7);
    vl->set(9);
    vl->set(7);
    vl->set(5);
    // now that we're back below speed (remember we said *above* 5), we can end
    // and look at details
    printf("The maximum when up to speed was %d, and the minimum was %d.\n", \
        vl->getmax(),vl->getmin());

    return 0;
}

CompilationError: test_value.cpp: In function ‘int main()’:
test_value.cpp:11:45: error: invalid use of non-static member function ‘void notifier::up_to_speed()’
     vl->abovelevel.connect(&ntfr, notifier::up_to_speed);
                                             ^
